# LangChain Basics

In [4]:
# installing the required libraries
!pip install -r ./requirements.txt -q

In [5]:
# !pip - installs the packages in the base environment
# pip - installs the packages in the virtual environment

In [6]:
pip show langchain

Name: langchain
Version: 0.0.233
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/mvp/anaconda3/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [9]:
# upgrading langchain
!pip install langchain --upgrade -q

#### Python-dotenv

In [10]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys (OpenAI, Pinecone) from .env
load_dotenv(find_dotenv(), override=True)

os.environ.get('PINECONE_API_KEY')

'19166-2a33-4aaf-bfd4-29736736536'

### LLM Models (Wrappers): GPT-3

In [11]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [12]:
output = llm('explain quantum mechanics in one sentence')
print(output)

AuthenticationError: Incorrect API key provided: sk-XR6cK******************************PaSH. You can find your API key at https://platform.openai.com/account/api-keys.

In [14]:
print(llm.get_num_tokens('explain quantum mechanics in one sentence'))

7


In [17]:
output = llm.generate(['... is the capital of France.', 
                   'What is the formula for the area of a circle?'])

In [18]:
print(output.generations)

[[Generation(text='\n\nParis.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where A is the area of the circle, π is the constant (approximately 3.14) and r is the radius of the circle.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [22]:
print(output.generations[0][0].text)



Paris.


In [23]:
len(output.generations)

2

In [27]:
output = llm.generate(['Write an orignal tagline for a burger restaurant'] * 3)

In [29]:
for o in output.generations:
    print(o[0].text, end='')



"Eat your heart out with our delicious burgers!"

"Taste the Difference: Gourmet Burgers from Our Grill to Your Plate"

"Taste the Difference with our Signature Burgers!"

### ChatModels: GPT-3.5-Turbo and GPT-4

In [30]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [31]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='explain quantum mechanics in one sentence')
]
output = chat(messages)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 81a6177dd62da29e1f60e1140f1ef3fa in your message.).


In [33]:
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf sehr kleinen Skalen und stellt dabei die klassische Vorstellung von deterministischen Prozessen in Frage.


### Prompt Templates

In [34]:
from langchain import PromptTemplate

In [35]:
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)
print(prompt)

input_variables=['virus', 'language'] output_parser=None partial_variables={} template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}.' template_format='f-string' validate_template=True


In [38]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='HIV', language='German'))
print(output)



HIV ist ein Virus, das das Immunsystem des Menschen schädigt und AIDS verursacht. Es wird hauptsächlich durch sexuellen Kontakt, Bluttransfusionen und den Austausch von Injektionsnadeln übertragen. HIV ist eine der weltweit größten Gesundheitsbedrohungen und hat seit Beginn der Pandemie weltweit viele Todesfälle verursacht.


### Simple Chains

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': 'HSV', 'language': 'french'})


In [46]:
print(output)

Le HSV, ou Herpes Simplex Virus, est un virus à ADN qui peut causer des infections cutanées et muqueuses chez les humains. Il existe deux types de HSV : le HSV-1, qui provoque généralement des lésions buccales et labiales, et le HSV-2, qui est principalement responsable des infections génitales. Bien que le HSV ne puisse pas être guéri, il peut être traité avec des médicaments antiviraux pour réduire la fréquence et la gravité des éruptions cutanées et des symptômes.


### Sequential Chains

In [51]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}.'''
)
chain1 = LLMChain(llm=llm1, prompt=prompt1)


llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('softmax')





> Entering new SimpleSequentialChain chain...


def softmax(x):
    """ 
    Computes softmax values for each sets of scores in x. 
    x: a numpy array of shape (N,) containing scores
    Returns: an array of shape (N,) containing the softmax scores for each entry of x
    """
    # Compute the exponentials of the input
    exp_x = np.exp(x)
    
    # Compute the sum of the exponentials
    sum_exp_x = np.sum(exp_x)
    
    # Compute softmax values
    softmax_x = exp_x / sum_exp_x
    
    return softmax_x
The given Python function, softmax(x) accepts a numpy array x of shape (N,) containing scores and returns an array of shape (N,) containing the softmax scores for each entry of x using the following formula:

softmax_x = exp_x/sum(exp_x)

where exp_x is the array obtained by taking the exponential of each entry of x.

First, the exponentials of each element of the numpy array x are computed using numpy's `exp` function and stored in a numpy array named exp_x. 

Next, the sum of

### LangChain Agents

In [55]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool  
from langchain.llms import OpenAI

In [57]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)
# agent_executor.run('Calculate the square root of the factorial of 20 \
# and display it with 4 decimal points')

agent_executor.run('what is the answer to 5.1 ** 7.3?')





> Entering new AgentExecutor chain...
 I need to use the power operator
Action: Python REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought: I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


'146306.05007233328'